# Fine-tuning LLaVA vision model: ViSC8

In [6]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from transformers import AutoModelForImageClassification, AutoTokenizer
from PIL import Image

## Object WebPageDataset
This object represents our dataset and creates utilities for working with the data.  
WebPageDataset inherits from __torch.utils.data.Dataset__.
### Parameters
- __csv_file:__ Path to an csv file.
- __root_dir:__ Path to the folder containing the images.
- __transform:__ A torchvision `transforms` object instance.

In [7]:
class WebPageDataset(Dataset):
    def __init__(self, csv_file:str, root_dir:str, transform=None):
        self.ratings_df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self) -> int:
        return len(self.ratings_df)

    def __getitem__(self, idx:int):
        img_name = os.path.join(self.root_dir, f"{self.ratings_df.iloc[idx, 0]}.png")
        image = Image.open(img_name).convert("RGB")
        rating = self.ratings_df.iloc[idx, 1]

        if self.transform: image = self.transform(image)
        return image, rating


In [8]:
DATA_DIR_IMGS:str = "./Dataset/comparison-based-dataset/images"
DATA_DIR_GRADES:str = "./Dataset/comparison-based-dataset/website_scores.csv"

TRANSFORM = transforms.Compose([ transforms.ToTensor()])

DATASET = WebPageDataset(DATA_DIR_GRADES, DATA_DIR_IMGS, TRANSFORM)
DATA_LOADER = DataLoader(DATASET, batch_size=32, shuffle=True)